In [1]:
R1=[1.5,1.51,1.52,1.53,1.54,1.55,1.56,1.57,1.58,1.59,1.6,1.61,1.62,1.63,1.64,1.65,1.66,1.67,1.68,1.69,1.7 ]

In [2]:
# import common packages
import numpy as np
from qiskit import Aer

# lib from Qiskit Aqua
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.components.optimizers import COBYLA


# lib from Qiskit Aqua Chemistry
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PSI4Driver
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry import MP2Info

In [3]:
    psi4_lih_config = '''
molecule mol {{
   0 1
   Li   0.0 0.0 0.0
   H   {} 0.0 0.0
   symmetry c1
}}

set {{
      basis sto-3g
      scf_type pk
      reference uhf
      guess huckel
      guess_mix true
      
}}
'''

In [4]:
map_type = 'parity'
qubit_reduction = True

In [13]:
HF_1=[]
nuclear_repulsion_energy1=[]
Ham1=[]
energy_shifts=[]
for i in range(len(R1)):
    length=R1[i]
    driver = PSI4Driver(config=psi4_lih_config.format(length)) 
    molecule=driver.run()
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2    
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    freeze_list = [0]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]    
    ferOp = FermionicOperator(h1=h1, h2=h2)
    if len(freeze_list) > 0:
        ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
        num_spin_orbitals -= len(freeze_list)
        num_particles -= len(freeze_list)    
    energy_shifts.append(energy_shift)
    nuclear_repulsion_energy1.append(molecule.nuclear_repulsion_energy)
    HF_1.append(molecule.hf_energy)
    qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) if qubit_reduction else qubitOp
    qubitOp.chop(10**-10)
    Ham1.append(qubitOp)

In [15]:
# setup COBYLA optimizer
max_eval = 2000
optimizer = COBYLA(maxiter=max_eval)

# setup HartreeFock state
HF_state = HartreeFock(num_orbitals=num_spin_orbitals, num_particles=num_particles,qubit_mapping=map_type,
                       two_qubit_reduction=True)


# setup UCCSD variational form
uccsd = UCCSD(num_orbitals=num_spin_orbitals, num_particles=num_particles, 
                 initial_state=HF_state, qubit_mapping=map_type,
                 two_qubit_reduction=qubit_reduction, num_time_slices=1)

# backend setting
backend = Aer.get_backend('statevector_simulator')

In [17]:
sto_uccsd=[]
sto_diagext=[]
for i in range(len(Ham1)):
    input_operator=Ham1[i]
    sto_diagext.append(float(NumPyEigensolver(input_operator).run().eigenvalues.real))
    vqe = VQE(operator=input_operator, optimizer=optimizer, quantum_instance=backend)
    vqe.var_form = uccsd
    result = vqe.compute_minimum_eigenvalue()
    sto_uccsd.append(result.eigenvalue.real)
    print('VQE:', result.eigenvalue.real)    

VQE: -1.1001883189906367
VQE: -1.0980070959139256
VQE: -1.0958177173452515
VQE: -1.0936208745913667
VQE: -1.0914171640719048
VQE: -1.0892072291553652
VQE: -1.0869916400864656
VQE: -1.0847710216289375
VQE: -1.0825458895006879
VQE: -1.0803168071228542
VQE: -1.0780842865315412
VQE: -1.075848852329038
VQE: -1.073610987870243
VQE: -1.0713711782112512
VQE: -1.0691298687968054
VQE: -1.066887534214747
VQE: -1.0646445940015195
VQE: -1.062401470685552
VQE: -1.0601585742651722
VQE: -1.0579163050043756
VQE: -1.0556750353243558


In [19]:
np.array(sto_uccsd)

[-1.1001883189906367,
 -1.0980070959139256,
 -1.0958177173452515,
 -1.0936208745913667,
 -1.0914171640719048,
 -1.0892072291553652,
 -1.0869916400864656,
 -1.0847710216289375,
 -1.0825458895006879,
 -1.0803168071228542,
 -1.0780842865315412,
 -1.075848852329038,
 -1.073610987870243,
 -1.0713711782112512,
 -1.0691298687968054,
 -1.066887534214747,
 -1.0646445940015195,
 -1.062401470685552,
 -1.0601585742651722,
 -1.0579163050043756,
 -1.0556750353243558]

In [29]:
import pandas as pd
HF=np.array(HF_1)-np.array(nuclear_repulsion_energy1)

dic = {
    "H-H Length": R1, 
    "HF": HF,
    "Ext_DIAG": sto_diagext,
    "UCCSD_VQE": sto_uccsd,
    "E_shift": energy_shifts,
    "VQE_result_shift": np.array(sto_uccsd)+np.array(energy_shifts),
    "Energy":np.array(sto_uccsd)+np.array(energy_shifts)+np.array(nuclear_repulsion_energy1)
}
df = pd.DataFrame(dic)
df

,H-H Length,HF,Ext_DIAG,UCCSD_VQE,E_shift,VQE_result_shift,Energy
0,1.50,-8.921712,-1.100188,-1.100188,-7.840306,-8.940494,-7.882140
1,1.51,-8.914727,-1.098007,-1.098007,-7.835631,-8.933638,-7.882293
2,1.52,-8.907793,-1.095818,-1.095818,-7.831019,-8.926837,-7.882408
3,1.53,-8.900909,-1.093621,-1.093621,-7.826468,-8.920088,-7.882486
4,1.54,-8.894075,-1.091417,-1.091417,-7.821976,-8.913393,-7.882529
5,1.55,-8.887289,-1.089207,-1.089207,-7.817543,-8.906751,-7.882537
6,1.56,-8.880552,-1.086992,-1.086992,-7.813168,-8.900160,-7.882511
7,1.57,-8.873862,-1.084771,-1.084771,-7.808849,-8.893620,-7.882454
8,1.58,-8.867219,-1.082546,-1.082546,-7.804585,-8.887131,-7.882365
9,1.59,-8.860622,-1.080317,-1.080317,-7.800376,-8.880693,-7.882245


### Select 9 pts and fitting to get $v$ : vibrational frequency

In [30]:
min_index=df[df.Energy==min(df['Energy'])].index.tolist()
min_index=min_index[0]

In [31]:
print(" bond length : ",df['H-H Length'][min_index])

 bond length :  1.55


In [32]:
fitting_E=df['Energy'][min_index-4:min_index+5].tolist()
fitting_length=df['H-H Length'][min_index-4:min_index+5]*1.8897259886
fitting_length.to_list()

[2.853486242786,
 2.872383502672,
 2.891280762558,
 2.910178022444,
 2.92907528233,
 2.947972542216,
 2.966869802102,
 2.985767061988,
 3.004664321874]

In [33]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

In [34]:
def f_1(x, A, B,C):
    return A * x**2 + B*x +C

In [35]:
z1 = np.polyfit(fitting_length,fitting_E,2) 
p_fit, prov = curve_fit(f_1, fitting_length, fitting_E)

In [36]:
curvature=z1[0]

In [38]:
H_mass=1.00782503223
Li_mass=7.0160034366
reduce_mass=1/(1/H_mass+1/Li_mass)

In [39]:
Hartree=(curvature*2/reduce_mass/1840)**0.5

In [40]:
v_f=Hartree*219474.63
v_f

1668.0504521762523